<a href="https://colab.research.google.com/github/BootCamp-BMA/colabs/blob/main/roberta_arabic_bert_arabertv2_withLoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/arabic_fake_news/FASSILA

02_experiment_results_DziriBERT.csv  experiment_results_MARABERT.csv
03_out.csv			     test_embeddings.csv
cleaned_data.csv		     test_set.csv
cleaned_test_set.csv		     train_embeddings.csv
cleaned_train_set.csv		     train_set.csv
cleaned_validation_set.csv	     validation_set.csv


In [ ]:
file_path = 'drive/MyDrive/arabic_fake_news/FASSILA/cleaned_data.csv'


In [ ]:
import pandas as pd

df = pd.read_csv(file_path)
print(df.head())


                                                news  label
0  فلقرن لواحد وعشرين لقاو الدوايات اللي ضد لفيروسات      1
1    عرف ردود الافعال عربيا وعالميا بعد واش صرا فغزة      0
2  راه معول مون فكوريا الجنوبية باش يتعاون مع روس...      0
3  تدعو ايرماراحشدا ل الواقعية المملكة المتحدة عل...      1
4                       الذهب طلع بدعم من عوامل فنية      0


In [ ]:
df.shape

(9636, 2)

In [ ]:
!pip uninstall -y sympy
!pip install sympy



Found existing installation: sympy 1.13.1
Uninstalling sympy-1.13.1:
  Successfully uninstalled sympy-1.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 37.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu12==12.4.127; platform_system 

In [ ]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [ ]:
import os
import time
import json
import random
import logging
import traceback
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from transformers.integrations import TensorBoardCallback

# === CONFIGURATION ===
file_path = 'drive/MyDrive/arabic_fake_news/FASSILA/cleaned_data.csv'
results_path = 'results.json'

models = [
    "aubmindlab/bert-base-arabertv2",
    "asafaya/bert-base-arabic",
    "xlm-roberta-base"
]

learning_rates = [1e-5, 2e-5, 5e-5]
batch_sizes = [8, 16]
num_epochs = [3, 4, 5]
dropouts = [0.1, 0.3]
n_experiments = 10  # number of unique experiments to try

# === LOGGING SETUP ===
logging.basicConfig(filename='log.txt', level=logging.INFO, format='%(asctime)s %(message)s')
logger = logging.getLogger()

# === LOAD DATA ===
df = pd.read_csv(file_path)
df = df[["news", "label"]].dropna()
df = df.rename(columns={"news": "text", "label": "label"})

train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# === TOKENIZATION FUNCTION ===
def tokenize(example, tokenizer):
    return tokenizer(example["text"], truncation=True, padding="max_length")

# === METRICS ===
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds),
        "roc_auc": roc_auc_score(labels, logits[:, 1]) if logits.shape[1] > 1 else 0.0,
    }

# === LOAD EXISTING RESULTS ===
if os.path.exists(results_path):
    with open(results_path, "r", encoding="utf-8") as f:
        previous_results = json.load(f)
else:
    previous_results = []

# Create a set of parameter keys for quick lookup
existing_experiment_keys = set(
    f"{res['model']}_{res['learning_rate']}_{res['batch_size']}_{res['epochs']}_{res['dropout']}"
    for res in previous_results
)

results = previous_results
tokenized_cache = {}
attempts = 0
successful_experiments = 0
max_attempts = 100  # Stop if too many failed/duplicate tries

# === RANDOM SEARCH EXPERIMENTS ===
while successful_experiments < n_experiments and attempts < max_attempts:
    attempts += 1

    # Random sample
    model_name = random.choice(models)
    lr = random.choice(learning_rates)
    batch_size = random.choice(batch_sizes)
    epochs = random.choice(num_epochs)
    dropout = random.choice(dropouts)

    exp_key = f"{model_name}_{lr}_{batch_size}_{epochs}_{dropout}"
    if exp_key in existing_experiment_keys:
        logger.info(f"Skipping duplicate experiment: {exp_key}")
        continue

    try:
        logger.info(f"🚀 Running new experiment {successful_experiments + 1}: {exp_key}")
        start_time = time.time()

        if model_name not in tokenized_cache:
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            tokenized_train = train_dataset.map(lambda x: tokenize(x, tokenizer), batched=True)
            tokenized_test = test_dataset.map(lambda x: tokenize(x, tokenizer), batched=True)
            tokenized_cache[model_name] = (tokenized_train, tokenized_test, tokenizer)
        else:
            tokenized_train, tokenized_test, tokenizer = tokenized_cache[model_name]

        # Load model with LoRA
        base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
        peft_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=8,
            lora_alpha=32,
            lora_dropout=dropout,
            bias="none"
        )
        model = get_peft_model(base_model, peft_config)

        output_dir = f"./checkpoints/{model_name.replace('/', '_')}_lr_{lr}_bs_{batch_size}_ep_{epochs}_do_{dropout}"
        os.makedirs(output_dir, exist_ok=True)

        training_args = TrainingArguments(
            output_dir=output_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            learning_rate=lr,
            logging_dir=f"./logs/{model_name.replace('/', '_')}",
            report_to="tensorboard",
            save_total_limit=1,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_test,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2), TensorBoardCallback()],
        )

        trainer.train()
        eval_result = trainer.evaluate()

        elapsed_time = time.time() - start_time
        logger.info(f"✅ Finished {exp_key} in {elapsed_time:.2f}s")

        new_result = {
            "experiment_id": len(results) + 1,
            "model": model_name,
            "learning_rate": lr,
            "batch_size": batch_size,
            "epochs": epochs,
            "dropout": dropout,
            "elapsed_time_sec": elapsed_time,
            "metrics": eval_result,
        }

        results.append(new_result)
        existing_experiment_keys.add(exp_key)
        successful_experiments += 1

        # Save results after every successful run
        with open(results_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

    except Exception as e:
        logger.error(f"❌ Error during {exp_key}: {e}")
        logger.error(traceback.format_exc())
        continue

logger.info("🎉 Random search completed.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-ae284220850e>:142: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoar

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Roc Auc
1,0.694600,0.675181,0.539699,0.132942,0.552846,0.075556,0.621785
2,0.687400,0.662710,0.537104,0.044968,0.617647,0.023333,0.628663
3,0.671200,0.653878,0.605086,0.618928,0.563355,0.686667,0.640359
4,0.666500,0.663587,0.565646,0.338340,0.586301,0.237778,0.653257
5,0.659800,0.647748,0.598858,0.526638,0.586630,0.477778,0.651859


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-ae284220850e>:142: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
You are adding a <class 'transformers.integrations.integration_utils.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback
EarlyStoppingCallback
No label_names provided for mo

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import json

with open("results.json", "r", encoding="utf-8") as f:
    results = json.load(f)

# Sort and show top 5 models by F1 score
sorted_results = sorted(results, key=lambda x: x['metrics']['eval_f1'], reverse=True)

for i, r in enumerate(sorted_results[:5], 1):
    print(f"{i}. {r['model']} | F1: {r['metrics']['eval_f1']:.4f} | Acc: {r['metrics']['eval_accuracy']:.4f} | Epochs: {r['epochs']} | LR: {r['learning_rate']}")


1. xlm-roberta-base | F1: 0.6189 | Acc: 0.6051 | Epochs: 5 | LR: 5e-05
2. asafaya/bert-base-arabic | F1: 0.5229 | Acc: 0.6051 | Epochs: 4 | LR: 2e-05
3. aubmindlab/bert-base-arabertv2 | F1: 0.3676 | Acc: 0.5625 | Epochs: 5 | LR: 1e-05
4. xlm-roberta-base | F1: 0.0000 | Acc: 0.5330 | Epochs: 5 | LR: 1e-05


In [ ]:
df_result = pd.DataFrame(results)

In [ ]:
df_result.shape

(4, 8)